In [1]:
import numpy as np, pandas as pd
import os,glob, re
%matplotlib inline

dfs = {re.search('([a-zA-Z_]*)\.csv', fn).group(1):pd.read_csv(fn) for fn in glob.glob(os.getcwd()+'//Raw//*.csv')}
print('data frames read:{}'.format(list(dfs.keys())))

print('local variables with the same names are created.')
for k, v in dfs.items(): locals()[k] = v

data frames read:['air_reserve', 'air_store_info', 'air_visit_data', 'date_info', 'hpg_store_info', 'sample_submission', 'store_id_relation']
local variables with the same names are created.


In [2]:
weekdayholidays = date_info.apply(lambda x: x.day_of_week in ['Saturday','Sunday'] and x.holiday_flg == 1,axis=1)
date_info.loc[weekdayholidays,'holiday_flg'] = 0
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,0
2,2016-01-03,Sunday,0
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [3]:
date_info['lagged'] = date_info['holiday_flg'].shift(-1)
day_b4_holiday = (date_info['holiday_flg'] == 0) & (date_info['lagged'] == 1.0) #& (date_info['day_of_week'] not ['Sunday','Saturday'])
date_info['dayb4holiday'] = 0
date_info.loc[day_b4_holiday,'dayb4holiday'] = 1
date_info.drop('lagged',axis = 1, inplace = True)
date_info.head()

,calendar_date,day_of_week,holiday_flg,dayb4holiday
0,2016-01-01,Friday,1,0
1,2016-01-02,Saturday,0,0
2,2016-01-03,Sunday,0,0
3,2016-01-04,Monday,0,0
4,2016-01-05,Tuesday,0,0


In [4]:
date_info['weights'] = (date_info.index + 1)/ len(date_info)
date_info.head()

,calendar_date,day_of_week,holiday_flg,dayb4holiday,weights
0,2016-01-01,Friday,1,0,0.001934
1,2016-01-02,Saturday,0,0,0.003868
2,2016-01-03,Sunday,0,0,0.005803
3,2016-01-04,Monday,0,0,0.007737
4,2016-01-05,Tuesday,0,0,0.009671


In [5]:
air_reserve['visit_datetime']  = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime']) 
#air_reserve['day_diff']  = (air_reserve['visit_datetime'] - air_reserve['reserve_datetime']).dt.days
air_reserve['reserved_before']  = abs(air_reserve['visit_datetime'] - air_reserve['reserve_datetime']).astype('timedelta64[h]')
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserved_before
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,3.0
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,0.0
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,0.0
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,4.0
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,19.0


In [6]:
def get_meal_type(hr):    
    if hr > 6 and hr <= 11:
        return 'breakfast'
    if hr >11 and hr <= 17:
        return 'lunch'
    if hr >17 and hr <= 23:
        return 'dinner'
    return 'midnight'
    
air_reserve['meal_type'] = air_reserve['visit_datetime'].dt.hour.apply(lambda x: get_meal_type(x))
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserved_before,meal_type
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1,3.0,dinner
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3,0.0,dinner
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6,0.0,dinner
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2,4.0,dinner
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5,19.0,dinner


In [7]:
air_reserve['visit_date'] = air_reserve.visit_datetime.dt.date.astype(str)
air_reserve.drop(['visit_datetime','reserve_datetime'],axis = 1 , inplace = True)
air_reserve.head()

,air_store_id,reserve_visitors,reserved_before,meal_type,visit_date
0,air_877f79706adbfb06,1,3.0,dinner,2016-01-01
1,air_db4b38ebe7a7ceff,3,0.0,dinner,2016-01-01
2,air_db4b38ebe7a7ceff,6,0.0,dinner,2016-01-01
3,air_877f79706adbfb06,2,4.0,dinner,2016-01-01
4,air_db80363d35f10926,5,19.0,dinner,2016-01-01


In [8]:
air_reserve = air_reserve.merge(date_info,how = 'left',left_on = 'visit_date',right_on = 'calendar_date')
air_reserve.drop(['calendar_date'],axis = 1, inplace = True)
air_reserve.head()

,air_store_id,reserve_visitors,reserved_before,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights
0,air_877f79706adbfb06,1,3.0,dinner,2016-01-01,Friday,1,0,0.001934
1,air_db4b38ebe7a7ceff,3,0.0,dinner,2016-01-01,Friday,1,0,0.001934
2,air_db4b38ebe7a7ceff,6,0.0,dinner,2016-01-01,Friday,1,0,0.001934
3,air_877f79706adbfb06,2,4.0,dinner,2016-01-01,Friday,1,0,0.001934
4,air_db80363d35f10926,5,19.0,dinner,2016-01-01,Friday,1,0,0.001934


In [9]:
air_reserve.tail()

,air_store_id,reserve_visitors,reserved_before,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights
92373,air_db4b38ebe7a7ceff,6,1007.0,dinner,2017-05-29,Monday,0,0,0.996132
92374,air_1033310359ceeac1,8,990.0,dinner,2017-05-30,Tuesday,0,0,0.998066
92375,air_877f79706adbfb06,3,1173.0,dinner,2017-05-31,Wednesday,0,0,1.000000
92376,air_900d755ebd2f7bbd,10,1012.0,dinner,2017-05-31,Wednesday,0,0,1.000000
92377,air_3cad29d1a23209d2,3,947.0,dinner,2017-05-31,Wednesday,0,0,1.000000


In [10]:
grouped = air_reserve.groupby(['air_store_id','meal_type','visit_date','day_of_week','holiday_flg','dayb4holiday','weights'])['reserve_visitors'].sum().reset_index()
grouped.head()

,air_store_id,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights,reserve_visitors
0,air_00a91d42b08b08d9,dinner,2016-10-31,Monday,0,0,0.589942,2
1,air_00a91d42b08b08d9,dinner,2016-12-05,Monday,0,0,0.657640,9
2,air_00a91d42b08b08d9,dinner,2016-12-14,Wednesday,0,0,0.675048,18
3,air_00a91d42b08b08d9,dinner,2016-12-17,Saturday,0,0,0.680851,2
4,air_00a91d42b08b08d9,dinner,2016-12-20,Tuesday,0,0,0.686654,4


In [11]:
grouped = grouped.merge(air_visit_data,how = 'left',on=['air_store_id','visit_date']).dropna()

In [12]:
grouped.head()

,air_store_id,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights,reserve_visitors,visitors
0,air_00a91d42b08b08d9,dinner,2016-10-31,Monday,0,0,0.589942,2,26.0
1,air_00a91d42b08b08d9,dinner,2016-12-05,Monday,0,0,0.657640,9,33.0
2,air_00a91d42b08b08d9,dinner,2016-12-14,Wednesday,0,0,0.675048,18,30.0
3,air_00a91d42b08b08d9,dinner,2016-12-17,Saturday,0,0,0.680851,2,17.0
4,air_00a91d42b08b08d9,dinner,2016-12-20,Tuesday,0,0,0.686654,4,37.0


In [13]:
t = grouped.groupby(['air_store_id','visit_date'])['meal_type'].count().reset_index()
t = t[t['meal_type'] > 1].drop('meal_type',axis  = 1)
t.head()

,air_store_id,visit_date
68,air_0164b9927d20bcc3,2017-03-01
89,air_0164b9927d20bcc3,2017-04-21
105,air_03963426c9312048,2016-01-09
111,air_03963426c9312048,2016-01-15
125,air_03963426c9312048,2016-01-30


In [14]:
grouped = grouped.merge(t,how = 'outer',on = ['air_store_id','visit_date'],indicator=True)
grouped.head()


,air_store_id,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights,reserve_visitors,visitors,_merge
0,air_00a91d42b08b08d9,dinner,2016-10-31,Monday,0,0,0.589942,2,26.0,left_only
1,air_00a91d42b08b08d9,dinner,2016-12-05,Monday,0,0,0.657640,9,33.0,left_only
2,air_00a91d42b08b08d9,dinner,2016-12-14,Wednesday,0,0,0.675048,18,30.0,left_only
3,air_00a91d42b08b08d9,dinner,2016-12-17,Saturday,0,0,0.680851,2,17.0,left_only
4,air_00a91d42b08b08d9,dinner,2016-12-20,Tuesday,0,0,0.686654,4,37.0,left_only


In [15]:
grouped.shape

(34901, 10)

In [16]:
grouped = grouped[grouped['_merge'] != 'both']
grouped.shape

(21847, 10)

In [17]:
grouped.drop('_merge',axis = 1,inplace = True)
grouped.head()

,air_store_id,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights,reserve_visitors,visitors
0,air_00a91d42b08b08d9,dinner,2016-10-31,Monday,0,0,0.589942,2,26.0
1,air_00a91d42b08b08d9,dinner,2016-12-05,Monday,0,0,0.657640,9,33.0
2,air_00a91d42b08b08d9,dinner,2016-12-14,Wednesday,0,0,0.675048,18,30.0
3,air_00a91d42b08b08d9,dinner,2016-12-17,Saturday,0,0,0.680851,2,17.0
4,air_00a91d42b08b08d9,dinner,2016-12-20,Tuesday,0,0,0.686654,4,37.0


In [18]:
grouped['showup_percent'] = grouped['reserve_visitors'] / grouped['visitors']
grouped.head()

,air_store_id,meal_type,visit_date,day_of_week,holiday_flg,dayb4holiday,weights,reserve_visitors,visitors,showup_percent
0,air_00a91d42b08b08d9,dinner,2016-10-31,Monday,0,0,0.589942,2,26.0,0.076923
1,air_00a91d42b08b08d9,dinner,2016-12-05,Monday,0,0,0.657640,9,33.0,0.272727
2,air_00a91d42b08b08d9,dinner,2016-12-14,Wednesday,0,0,0.675048,18,30.0,0.600000
3,air_00a91d42b08b08d9,dinner,2016-12-17,Saturday,0,0,0.680851,2,17.0,0.117647
4,air_00a91d42b08b08d9,dinner,2016-12-20,Tuesday,0,0,0.686654,4,37.0,0.108108


In [19]:
weighted_mean = lambda x : ((x.showup_percent * x.weights).sum() / (x.weights).sum())
visitors_per_weekday = grouped.groupby(['air_store_id','meal_type','day_of_week','holiday_flg','dayb4holiday']).apply(weighted_mean).reset_index()

In [20]:
visitors_per_weekday.head()

,air_store_id,meal_type,day_of_week,holiday_flg,dayb4holiday,0
0,air_00a91d42b08b08d9,dinner,Monday,0,0,0.180138
1,air_00a91d42b08b08d9,dinner,Saturday,0,0,0.324526
2,air_00a91d42b08b08d9,dinner,Thursday,0,0,0.300000
3,air_00a91d42b08b08d9,dinner,Tuesday,0,0,0.123772
4,air_00a91d42b08b08d9,dinner,Wednesday,0,0,0.367195


In [21]:
sample_submission["airstore"],sample_submission["date"] = sample_submission.id.str[:20],sample_submission.id.str[21:]
sample_submission.head()

,id,visitors,airstore,date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [22]:
combined = sample_submission.merge(
    air_reserve.groupby(['air_store_id','meal_type','day_of_week','holiday_flg','dayb4holiday','visit_date'])['reserve_visitors'].sum().reset_index(),
    left_on = ['airstore','date'],right_on = ['air_store_id','visit_date'])

In [23]:
combined = combined.merge(visitors_per_weekday,how = 'left', on=['air_store_id','meal_type','day_of_week','holiday_flg','dayb4holiday'])

In [24]:
combined.rename(columns = {0:'ratio'},inplace = True)

In [25]:
combined.head()

,id,visitors,airstore,date,air_store_id,meal_type,day_of_week,holiday_flg,dayb4holiday,visit_date,reserve_visitors,ratio
0,air_0164b9927d20bcc3_2017-04-25,0,air_0164b9927d20bcc3,2017-04-25,air_0164b9927d20bcc3,dinner,Tuesday,0,0,2017-04-25,3,0.599653
1,air_0164b9927d20bcc3_2017-04-26,0,air_0164b9927d20bcc3,2017-04-26,air_0164b9927d20bcc3,lunch,Wednesday,0,0,2017-04-26,2,NaN
2,air_0164b9927d20bcc3_2017-04-27,0,air_0164b9927d20bcc3,2017-04-27,air_0164b9927d20bcc3,dinner,Thursday,0,0,2017-04-27,4,0.759742
3,air_0164b9927d20bcc3_2017-05-01,0,air_0164b9927d20bcc3,2017-05-01,air_0164b9927d20bcc3,dinner,Monday,0,0,2017-05-01,3,0.745376
4,air_0164b9927d20bcc3_2017-05-02,0,air_0164b9927d20bcc3,2017-05-02,air_0164b9927d20bcc3,dinner,Tuesday,0,1,2017-05-02,2,0.900000


In [26]:
combined['_visitors'] = combined['reserve_visitors'] /combined['ratio']

In [27]:
combined = combined[['id','_visitors']]

In [28]:
combined = combined.groupby('id')['_visitors'].sum().reset_index()

In [29]:
combined.shape

(1131, 2)

In [30]:
combined.dropna(inplace = True)

In [31]:
sample_submission.head()

,id,visitors,airstore,date
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27


In [32]:
sample_submission.shape

(32019, 4)

In [33]:
sample_submission = sample_submission.merge(air_visit_data.groupby('air_store_id')['visitors'].max().reset_index(),
                        left_on = 'airstore',right_on = 'air_store_id')
sample_submission.head()
#sample_submission.drop(['visitors_x','air_store_id'],axis = 1, inplace =True)

,id,visitors_x,airstore,date,air_store_id,visitors_y
0,air_00a91d42b08b08d9_2017-04-23,0,air_00a91d42b08b08d9,2017-04-23,air_00a91d42b08b08d9,99
1,air_00a91d42b08b08d9_2017-04-24,0,air_00a91d42b08b08d9,2017-04-24,air_00a91d42b08b08d9,99
2,air_00a91d42b08b08d9_2017-04-25,0,air_00a91d42b08b08d9,2017-04-25,air_00a91d42b08b08d9,99
3,air_00a91d42b08b08d9_2017-04-26,0,air_00a91d42b08b08d9,2017-04-26,air_00a91d42b08b08d9,99
4,air_00a91d42b08b08d9_2017-04-27,0,air_00a91d42b08b08d9,2017-04-27,air_00a91d42b08b08d9,99


In [35]:
sample_submission.drop(['visitors_x','air_store_id'],axis = 1, inplace =True)

In [37]:
sample_submission.rename(columns = {'visitors_y':'max_cap'},inplace = True)
sample_submission.head()

,id,airstore,date,max_cap
0,air_00a91d42b08b08d9_2017-04-23,air_00a91d42b08b08d9,2017-04-23,99
1,air_00a91d42b08b08d9_2017-04-24,air_00a91d42b08b08d9,2017-04-24,99
2,air_00a91d42b08b08d9_2017-04-25,air_00a91d42b08b08d9,2017-04-25,99
3,air_00a91d42b08b08d9_2017-04-26,air_00a91d42b08b08d9,2017-04-26,99
4,air_00a91d42b08b08d9_2017-04-27,air_00a91d42b08b08d9,2017-04-27,99


In [122]:
sss = pd.read_csv('sample_submission444.csv')
sss.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,2.0000
1,air_00a91d42b08b08d9_2017-04-24,24.6557
2,air_00a91d42b08b08d9_2017-04-25,29.6820
3,air_00a91d42b08b08d9_2017-04-26,28.0039
4,air_00a91d42b08b08d9_2017-04-27,33.0636


In [123]:
sss = sss.merge(combined, how = 'left',on = 'id')

In [124]:
sss.head()

,id,visitors,_visitors
0,air_00a91d42b08b08d9_2017-04-23,2.0000,NaN
1,air_00a91d42b08b08d9_2017-04-24,24.6557,NaN
2,air_00a91d42b08b08d9_2017-04-25,29.6820,NaN
3,air_00a91d42b08b08d9_2017-04-26,28.0039,NaN
4,air_00a91d42b08b08d9_2017-04-27,33.0636,NaN


In [125]:
sss.rename(columns = {'_visitors': 'w_visitors'},inplace = True)
sss.head()

,id,visitors,w_visitors
0,air_00a91d42b08b08d9_2017-04-23,2.0000,NaN
1,air_00a91d42b08b08d9_2017-04-24,24.6557,NaN
2,air_00a91d42b08b08d9_2017-04-25,29.6820,NaN
3,air_00a91d42b08b08d9_2017-04-26,28.0039,NaN
4,air_00a91d42b08b08d9_2017-04-27,33.0636,NaN


In [126]:
sss = sss.merge(sample_submission, on = ['id'],how = 'outer')

In [127]:
sss.shape

(32019, 6)

In [128]:
sss.head()

,id,visitors,w_visitors,airstore,date,max_cap
0,air_00a91d42b08b08d9_2017-04-23,2.0000,NaN,air_00a91d42b08b08d9,2017-04-23,99
1,air_00a91d42b08b08d9_2017-04-24,24.6557,NaN,air_00a91d42b08b08d9,2017-04-24,99
2,air_00a91d42b08b08d9_2017-04-25,29.6820,NaN,air_00a91d42b08b08d9,2017-04-25,99
3,air_00a91d42b08b08d9_2017-04-26,28.0039,NaN,air_00a91d42b08b08d9,2017-04-26,99
4,air_00a91d42b08b08d9_2017-04-27,33.0636,NaN,air_00a91d42b08b08d9,2017-04-27,99


In [129]:
sss.loc[sss['w_visitors'] > sss['max_cap'],'w_visitors'] = sss['max_cap']

In [130]:
sss.drop(['airstore','date','max_cap'],inplace = True,axis = 1)
sss.head()

,id,visitors,w_visitors
0,air_00a91d42b08b08d9_2017-04-23,2.0000,NaN
1,air_00a91d42b08b08d9_2017-04-24,24.6557,NaN
2,air_00a91d42b08b08d9_2017-04-25,29.6820,NaN
3,air_00a91d42b08b08d9_2017-04-26,28.0039,NaN
4,air_00a91d42b08b08d9_2017-04-27,33.0636,NaN


In [131]:
sss[pd.notnull(sss['w_visitors'])]

,id,visitors,w_visitors
41,air_0164b9927d20bcc3_2017-04-25,6.6159,5.002895
43,air_0164b9927d20bcc3_2017-04-27,6.8930,5.264944
47,air_0164b9927d20bcc3_2017-05-01,6.2326,4.024812
48,air_0164b9927d20bcc3_2017-05-02,6.6159,2.222222
55,air_0164b9927d20bcc3_2017-05-09,6.6159,5.002895
274,air_03963426c9312048_2017-04-24,24.1174,10.060180
276,air_03963426c9312048_2017-04-26,20.3600,32.461919
277,air_03963426c9312048_2017-04-27,24.6951,2.546777
278,air_03963426c9312048_2017-04-28,65.2155,35.848791
279,air_03963426c9312048_2017-04-29,57.1970,31.167197


In [82]:
sss.loc[pd.notnull(sss['w_visitors']),'visitors'] = (sss['visitors'] +sss['w_visitors']) / 2
sss.drop('w_visitors',inplace = True,axis = 1)
sss.to_csv('sample_submission555.csv', float_format='%.4f', index=None)
### error  = .499

,id,visitors,w_visitors
41,air_0164b9927d20bcc3_2017-04-25,5.809397,5.002895
43,air_0164b9927d20bcc3_2017-04-27,6.078972,5.264944
47,air_0164b9927d20bcc3_2017-05-01,5.128706,4.024812
48,air_0164b9927d20bcc3_2017-05-02,4.419061,2.222222
55,air_0164b9927d20bcc3_2017-05-09,5.809397,5.002895
274,air_03963426c9312048_2017-04-24,17.088790,10.060180
276,air_03963426c9312048_2017-04-26,26.410959,32.461919
277,air_03963426c9312048_2017-04-27,13.620939,2.546777
278,air_03963426c9312048_2017-04-28,50.532146,35.848791
279,air_03963426c9312048_2017-04-29,44.182099,31.167197


In [97]:
sss.loc[pd.notnull(sss['w_visitors']),'visitors'] = 1/((1/sss['visitors']) +(1/sss['w_visitors']))
sss.drop('w_visitors',inplace = True,axis = 1)
sss.to_csv('sample_submission666.csv', float_format='%.4f', index=None)
#.607

In [109]:
sss.loc[pd.notnull(sss['w_visitors']),'visitors'] = (0.3 * sss['visitors']) + (.7 * sss['w_visitors'])
sss.drop('w_visitors',inplace = True,axis = 1)
sss.to_csv('sample_submission777.csv', float_format='%.4f', index=None)
#.510

In [121]:
sss.loc[pd.notnull(sss['w_visitors']),'visitors'] = sss['w_visitors']
sss.drop('w_visitors',inplace = True,axis = 1)
sss.to_csv('sample_submission888.csv', float_format='%.4f', index=None)
#.553

In [132]:
sss.loc[pd.notnull(sss['w_visitors']),'visitors'] = (0.7 * sss['visitors']) + (.3 * sss['w_visitors'])
sss.drop('w_visitors',inplace = True,axis = 1)
sss.to_csv('sample_submission999.csv', float_format='%.4f', index=None)
#.510